<b> 
Takes existing BQ PROC TEMPLATE and inserts SQL from [referenced external files defined by "Analysts"] while replacing placeholders with PROC PARAMETERS AND VARIABLES:

    ...
    


In [32]:
import sys
import os

# load up a cutdown version of Paul's FileOps class from dep subfolder
if 'Documents\ipynb\dep' not in os.getcwd():
    os.chdir('dep')

from FileOps import fs

bqfp = os.path.join('C:','\git', 'index2', 'bq', 'r', 'sp')
bqfpStash = os.path.join('C:','\git', 'gitMyStash', 'af', 'dags', 'sq')

In [33]:
#X3
#tpl = 'iata_2019.sp_process_X3_TEMPLATE.sql'
#sqy = 'X3.sql'

In [34]:
#I1
#tpl = 'iata.sp_process_I1_TEMPLATE.sql'
#sqy = 'I1_2019.sql'

In [35]:
#I2
tpl = 'iata.sp_process_I2_TEMPLATE.sql'
sqy = 'I2_2019.sql'

In [267]:
#I3
#tpl = 'iata.sp_process_I3_TEMPLATE.sql'
#sqy = 'I3_2019.sql'

In [268]:
#X6
#tpl = 'matching.sp_process_X6_TEMPLATE.sql'
#sqy = 'X6_2019.sql'

In [269]:
#X7
#tpl = 'index.sp_process_X7_TEMPLATE.sql'
#sqy = 'X7_2019.sql'

In [36]:
spfp = os.path.join(bqfp, tpl)

#We use gitMyStash for Others code to avoid accidentally commiting Others code in our branch
qyfp = os.path.join(bqfpStash, sqy)

# recently changed spOut to goto gitStash so can WinCompare output with index2 first
spOut= os.path.join(bqfpStash, tpl).replace('_TEMPLATE','')

# first we open the sp Template
spt = fs()
spStr = spt.getLinesFromFile(spfp)

In [37]:
spLineList = spStr.split('\n')

In [38]:
# loop through lines until hit 
#   [MAIN and then split that line into it's components and record listIndex
#   [--%%INSERT%%] and record listIndex
for i, s in enumerate(spLineList):
    if '[MAIN' in s:
        aMain = s.strip()[1:-1].split(' | ')
        iMain = i
    elif '[--%%INSERT%%]' in s:
        iInsert = i

In [39]:
# open the pipeline.SQL specified in '[MAIN' placeholder in sp Template
qyt = fs()
qyStr= qyt.getLinesFromFile(qyfp) #aMain[1])

In [40]:
# loop through the replace placeholders defined inside '[MAIN'
for j in aMain[2:]:
    aPair = j.split(',')
    qyStr = qyStr.replace(aPair[0],aPair[1])

In [41]:
#  split by ), and REPLACE 
#    FIRST line [1,-X] with CREATE TEMP TABLE
#    LAST  line , with ;
qyStatementList = qyStr.replace('\n),\n','\n);\n|||').split('|||')

qyListNew = []
qyListNew.append(qyStatementList[0].replace('WITH','CREATE OR REPLACE TEMP TABLE') + '\n\n')

for i2, s2 in enumerate(qyStatementList[1:]):
    qyListNew.append('CREATE TEMP TABLE ' + s2.lstrip() + '\n\n')

In [42]:
# when using "named subqueries" ( WITH AS () ) the last subquery does not have a ',' so was not split...
#  ...so further split qyListNew[-1] by --%%INSERT%% to split the last subquery from the last SELECT used to INSERT
qyStrLast2 = qyListNew[-1].split('--%%INSERT%%')

In [43]:
#Now that we have treated and split Last2 SQL Statements, need to delete/pop the pre-split and replace with splits
qyListNew.pop()
qyListNew.append(qyStrLast2[0].rstrip() + ';\n')
qyListNew.append(qyStrLast2[1].strip() + ';')

In [44]:
# rejoin our fixed pipeline.SQL
sNew = ''.join(qyListNew[:-1])

In [279]:
#qyListNew[-2]
#This last query can lose ); at end when doco between last 2 statements  eg. \n)\n\n-- Main query...
#If consistent pattern can dealt with it here but for now handle manually

In [45]:
# insert non-INSERT and INSERT sections of pipeline.SQL into our sp
spLineList[iMain] = sNew
spLineList[iInsert] = qyListNew[-1]

In [46]:
# our new proc is ready as a string so write it!
spNew = '\n'.join(spLineList)
open(spOut, 'w').write(spNew)

4100